# Import modules

In [4]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

# Set variables

In [5]:
url = 'https://notifications.hse.gov.uk/COMAH2015/Search.aspx'
data = {'ctl00$ContentPlaceHolder1$Postcode':'M27 5AP',
        'ctl00$ContentPlaceHolder1$btnSearch': 'Search'}

# Define functions

## Submit information to Form in URL

In [11]:
def submit_form(url,data):
    response = requests.get(url)
    
    soup = bs(response.content)
    data['__VIEWSTATE'] = soup.select_one('#__VIEWSTATE')['value']
    data['__VIEWSTATEGENERATOR'] = soup.select_one('#__VIEWSTATEGENERATOR')['value']
    data['__EVENTVALIDATION'] = soup.select_one('#__EVENTVALIDATION')['value']

    response = requests.post(url,data=data)#,headers=headers)
    doc = bs(response.text, 'html.parser')
    return doc

In [13]:
doc = submit_form(url,data)

In [32]:
# with requests.Session() as s:
#     response = s.get(url)

#     soup = bs(response.content)
#     data['__VIEWSTATE'] = soup.select_one('#__VIEWSTATE')['value']
#     data['__VIEWSTATEGENERATOR'] = soup.select_one('#__VIEWSTATEGENERATOR')['value']
#     data['__EVENTVALIDATION'] = soup.select_one('#__EVENTVALIDATION')['value']
    
#     response = s.post(url,data=data,headers=headers)
#     doc = bs(response.text, 'html.parser')

# Peform check

In [43]:
def check_results_page(doc):
    if 'COMAH 2015: Search' in doc.text:
        res = 'ERROR - page is "Search"'
    elif 'COMAH 2015: Results' in doc.text:   
        res = 'OK - page is "Results"'
    else: 
        res = 'error'
    return res

In [45]:
check_results_page(doc)

'OK - page is "Results"'

# Get links from results page

In [46]:
def get_links_from_results(result_html):
    res=[]
    for link_element in doc.find_all('a'):
        link = link_element.get('href')
        if link != None and 'PublicInformation' in link:
            link = 'https://notifications.hse.gov.uk/comah2015/'+link
            res.append(link)
    return res

In [47]:
links = get_links_from_results(doc)

# Extract tables from links

continue with a single 'response' for now...

In [48]:
response = requests.get(links[0])

In [49]:
doc = bs(response.text,'html.parser')

In [50]:
# find tab id and corresponding name for given link
table_names = {}
for tab_header in doc.find("div", {"class": "ajax__tab_header"}):
    try:    
        tab_id = tab_header.get('id').rsplit('_',1)[0]
        table_name = tab_header.get_text()
        table_names[tab_id] = table_name
        a = tab_id
    except: pass

In [51]:
table_names

{'ContentPlaceHolder1_tcInfo_tpPanel1': 'Establishment Information',
 'ContentPlaceHolder1_tcInfo_tpPanel2': 'Substances and Emergency Information',
 'ContentPlaceHolder1_tcInfo_tpPanel3': 'Accident Hazards and Scenarios',
 'ContentPlaceHolder1_tcInfo_tpPanel4': 'Consequences of Accident',
 'ContentPlaceHolder1_tcInfo_tpPanel5': 'About COMAH Public Information'}

In [52]:
for div in doc.find_all('div',{'class':'ajax__tab_panel'}):
    print(div.get('id'))
    table_html = div.find('table')

ContentPlaceHolder1_tcInfo_tpPanel1
ContentPlaceHolder1_tcInfo_tpPanel2
ContentPlaceHolder1_tcInfo_tpPanel3
ContentPlaceHolder1_tcInfo_tpPanel4
ContentPlaceHolder1_tcInfo_tpPanel5


In [53]:
for table in doc.find_all('table'):
    print('-'*100)
    print(table.find_all('td'))
#     print(table_names)
#     print('-'*100)
#     for row in table.find_all('tr'):
#         print(row)

----------------------------------------------------------------------------------------------------
[<td class="db" colspan="2">
<strong>Operator</strong>
</td>, <td class="db" width="25%">
                           Operator Name
                        </td>, <td class="dbdata">
<span id="ContentPlaceHolder1_tcInfo_tpPanel1_lblLegalName"></span>
</td>, <td class="db">
                           Trading As
                        </td>, <td class="dbdata">
<span id="ContentPlaceHolder1_tcInfo_tpPanel1_lblTradeName"></span>
</td>, <td colspan="2">
                            
                        </td>, <td class="db" colspan="2">
<strong>Address</strong>
</td>, <td class="db">
                           Establishment Name
                        </td>, <td class="dbdata">
<span id="ContentPlaceHolder1_tcInfo_tpPanel1_lblEstablishmentName"></span>
</td>, <td class="db" style="vertical-align: top">
                           Address
                        </td>, <td class="dbdata">

# Main program

In [ ]:
# for n,form in enumerate(forms,start=1):
#     print('Form',n,'input elements')
#     print('type',' '*6,'attributes',' '*40,'name','\n'+'-'*100)
#     for i in form.find_all("input"):
#         input_type = i['type']
#         input_attrs = list(i.attrs.keys())
#         s1 = input_type
#         s2 = input_attrs
#         s3 = i['name']
#         pad12 = ' '*(10-len(s1))
#         pad23 = ' '*(50-(len(str(s2))))
#         print(s1,pad12,s2,pad23,s3)